In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_community.chat_models import ChatOllama
from typing import TypedDict
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

In [14]:
load_dotenv()

model = ChatOllama(model="deepseek-r1:latest", temperature=0)

In [15]:
class BlogState(TypedDict):

    title:str
    outline:str
    content:str

In [16]:
def create_outline(state:BlogState) -> BlogState:

    title = state['title']

    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    state['outline'] = outline # type:ignore

    return state

def create_content(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content # type:ignore

    return state

In [17]:
graph = StateGraph(BlogState)

# Add Nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_content', create_content)

# Add Edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_content')
graph.add_edge('create_content', END)

# Compile
workflow = graph.compile()

In [ ]:
# Excute
initial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(initial_state) # type:ignore

print(final_state['answer'])